In [ ]:
import os

os.getcwd()

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="eGjHxmLCAIMYJ89pbtFB")
project = rf.workspace("eeetan").project("doors-oyetk-kfpop-gv5s4")
version = project.version(1)
dataset = version.download("yolov11")

In [1]:
from ultralytics import YOLO

In [2]:
model = YOLO('yolo11n.pt')

In [ ]:
results = model.train(data="C://Users/Eeeta/Geesehacks/hyacinthe/model_training/doors-1/data.yaml", epochs=20, imgsz=640,batch=8)

In [2]:
trained = YOLO('sign.pt')

In [7]:
import os
from ultralytics import YOLO
import cv2
import pytesseract

# Open the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

ids = set()

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture image.")
        break

    # Run YOLO inference on the frame
    results = trained.track(frame, persist=True, verbose=False, tracker='bytetrack.yaml', show=True, conf=0.65)

    # Visualize the results on the frame
    annotated_frame = results[0].plot()  # YOLOv8 has built-in plotting for results

    # Process each detected object
    for result in results[0].boxes:
        # Extract bounding box, confidence, and class
        box = result.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2]
        conf = result.conf.cpu().item()     # Confidence score
        cls = int(result.cls.cpu().item())  # Class index
        print(result)
        if(result.id is not None):
            if(result.id not in ids):
                id = int(result.id.cpu().item())

                # Crop the frame around the detected object
                x1, y1, x2, y2 = map(int, box)
                cropped_object = frame[y1:y2, x1:x2]

                # Ensure dimensions are valid before extracting text
                if cropped_object.size > 0:
                    # Extract text directly from the cropped object
                    text = pytesseract.image_to_string(cropped_object)
                    print(f"Text extracted from detected object:\n{text.strip()}")

                ids.add(id)

    # Display the annotated frame
    cv2.imshow("YOLO11 Tracking", annotated_frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture and close the window
cap.release()
cv2.destroyAllWindows()


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.7715])
data: tensor([[1.4088e+03, 1.1179e+00, 1.5985e+03, 1.0756e+02, 2.0000e+01, 7.7154e-01, 0.0000e+00]])
id: tensor([20.])
is_track: True
orig_shape: (1080, 1920)
shape: torch.Size([1, 7])
xywh: tensor([[1503.6617,   54.3391,  189.6680,  106.4422]])
xywhn: tensor([[0.7832, 0.0503, 0.0988, 0.0986]])
xyxy: tensor([[1.4088e+03, 1.1179e+00, 1.5985e+03, 1.0756e+02]])
xyxyn: tensor([[0.7338, 0.0010, 0.8325, 0.0996]])
Text extracted from detected object:
LAI]
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.8481])
data: tensor([[1.2256e+03, 0.0000e+00, 1.4048e+03, 1.9637e+02, 8.4808e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1315.1904,   98.1843,  179.1757,  196.3685]])
xywhn: tensor([[0.6850, 0.0909, 0.0933, 0.1818]])
xyxy: tensor([[1225.6025,    0.0000, 1404.7782,  196.3685]])
xyxyn: tenso

KeyboardInterrupt: 